In [ ]:
!apt install libnvrtc8.0
!pip uninstall -y mxnet-cu80 && pip install mxnet-cu80==1.1.0
!pip install turicreate

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274.9 MB 4.0 kB/s 
     |████████████████████████████████| 5.0 MB 42.9 MB/s 
     |████████████████████████████████| 88 kB 9.1 MB/s 
     |████████████████████████████████| 132 kB 70.0 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  ERROR: Failed cleaning build dir for numpy
Failed to build numpy
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: idna
    Found existing installation: idna 2.10
 

In [ ]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
import sklearn.model_selection as model_selection
# sys.path is a built-in variable within the sys module. It contains a list of directories that the interpreter will search in for the required module.
import sys
sys.path.append("..")
from sklearn.model_selection import train_test_split


In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# mounting drive to easily access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing files from drive
customers = pd.read_csv('/content/drive/MyDrive/data/recommend_1.csv')
transactions = pd.read_csv('/content/drive/MyDrive/data/transactions.csv' )

In [ ]:

# shape indicates the number of rows and columns; head function is used to print the first five records
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [ ]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


Data Preparation

Our goal here is to break down each list of items in the products column into rows and count the number of products bought by a user

In [ ]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(6).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0
2,2,111.0,107.0,29.0,11.0,11.0,11.0,33.0,23.0,NaN,NaN
3,3,164.0,227.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,144.0,144.0,55.0,266.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(),
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

In [ ]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


 Create dummy

In [ ]:
# Dummy for marking whether a customer bought that item or not.
# If one buys an item, then purchase_dummy are marked as 1
# Why create a dummy instead of normalizing it..Normalizing the purchase count, for example by each user,
# would not work because customers may have different buying frequency ;don’t have the same taste.
# However, we can normalize items by purchase frequency across all users
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

Normalize

In [ ]:
# we will normalize purchase frequency of each item across users by first creating a user-item matrix
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
(df_matrix.shape)

(24429, 300)

In [ ]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# create a table that will be used as an input to the modeling
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


Splitting Training and Testing set

In [ ]:
#Split train and test set
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [ ]:
## splitting for both dummy table and scaled/normalized purchase table also
train_data_dummy, test_data_dummy = train_test_split(data_dummy)
train_data_norm, test_data_norm = train_test_split(data_norm)

In [ ]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part
## SFrame means scalable data frame. A tabular, column-mutable dataframe object that can scale to big data.
## The data in SFrame is stored column-wise, and is stored on persistent storage (e.g. disk) to avoid being constrained by memory size.
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)
train_data_dummy = tc.SFrame(train_data_dummy)
test_data_dummy = tc.SFrame(test_data_dummy)
train_data_norm = tc.SFrame(train_data_norm)
test_data_norm = tc.SFrame(test_data_norm)

In [ ]:
train_data

customerId,productId,purchase_count
7071,2,2
20021,115,1
20507,225,1
1666,20,1
6031,14,1
10720,286,1
4166,291,2
10375,23,1
4095,179,2
1244,21,1


In [ ]:
test_data

customerId,productId,purchase_count
934,42,1
16060,139,3
22038,129,2
25155,176,1
21738,1,2
18144,66,1
15608,39,1
22218,8,3
17048,84,1
5191,57,1


Popularity Model

In [ ]:
# The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
## We use turicreate library for running and evaluating both baseline and collaborative filtering models below
# Training data is used for model selection

user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [ ]:
# Create a model that makes recommendations using item popularity
popularity_model = tc.popularity_recommender.create(train_data,
                                                    user_id=user_id,
                                                    item_id=item_id,
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23257 users and 300 items.

Data prepared in: 0.130203s

106868 observations to process; with 300 unique items.

In [ ]:
# top 30 rows for first 3 customers with 10 recommendations each will be printed

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 117110

recommendations finished on 2000/62483 queries. users per second: 114929

recommendations finished on 3000/62483 queries. users per second: 116009

recommendations finished on 4000/62483 queries. users per second: 119193

recommendations finished on 5000/62483 queries. users per second: 116689

...

recommendations finished on 59000/62483 queries. users per second: 107673

recommendations finished on 60000/62483 queries. users per second: 107679

recommendations finished on 61000/62483 queries. users per second: 107837

recommendations finished on 62000/62483 queries. users per second: 107550

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |     34    | 3.066945606694561  |  1   |
|     0      |     37    | 3.0116731517509727 |  2   |
|     0      |    132    | 2.981132075471698  |  3   |
|     0      |     0     | 2.956813819577735  |  4   |
|     0      |    248    | 2.951219512195122  |  5   |
|     0      |     3     | 2.8270833333333334 |  6   |
|     0      |    110    |        2.7         |  7   |
|     0      |     27    | 2.656716417910448  |  8   |
|     0      |     32    | 2.6142857142857143 |  9   |
|     0      |     10    | 2.6064139941690962 |  10  |
|     1      |     34    | 3.066945606694561  |  1   |
|     1      |     37    | 3.0116731517509727 |  2   |
|     1      |    132    | 2.981132075471698  |  3   |
|     1      |     0     | 2.956813819577735  |  4   |
|     1      |    248    | 2.951219512195122  |  5   |
|     1   

In [ ]:
# Since turicreate is very accessible library, we can define a model selection function as below
# model type = popularity
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data,
                                                    user_id=user_id,
                                                    item_id=item_id,
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data,
                                                    user_id=user_id,
                                                    item_id=item_id,
                                                    target=target,
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data,
                                                    user_id=user_id,
                                                    item_id=item_id,
                                                    target=target,
                                                    similarity_type='pearson')

    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model


In [ ]:
# these variables will change accordingly, we have used trained dummy data i.e. puchase dummy

name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 100188 observations with 22999 users and 300 items.

Data prepared in: 0.136852s

...

recommendations finished on 59000/62483 queries. users per second: 110642

recommendations finished on 60000/62483 queries. users per second: 110384

recommendations finished on 61000/62483 queries. users per second: 110666

recommendations finished on 62000/62483 queries. users per second: 110531

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |    131    |  1.0  |  1   |
|     0      |    276    |  1.0  |  2   |
|     0      |    178    |  1.0  |  3   |
|     0      |    296    |  1.0  |  4   |
|     0      |    218    |  1.0  |  5   |
|     0      |    133    |  1.0  |  6   |
|     0      |    149    |  1.0  |  7   |
|     0      |    221    |  1.0  |  8   |
|     0      |     2     |  1.0  |  9   |
|     0      |     16    |  1.0  |  10  |
|     1      |     11    |  1.0  |  1   |
|     1      |    271    |  1.0  |  2   |
|     1      |    131    |  1.0  |  3   |
|     1      |    276    |  1.0  |  4   |
|     1      |    178    |  1.0  |  5   |
|     1      |    218    |  1.0  |  6   |
|     1      |    133    |  1.0  |  7   |
|     1      |    149    |  1.0  |  8   |
|     1      |    221    |  1.0  |  9   |
|     1      |     16    |  1.0  |  10  |
|     2      |    131    |  1.0  |

In [ ]:
# we have used trained normalized data here

name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 100188 observations with 22877 users and 300 items.

Data prepared in: 0.134255s

100188 observations to process; with 300 unique items.

recommendations finished on 1000/62483 queries. users per second: 65647

...

recommendations finished on 59000/62483 queries. users per second: 99715.2

recommendations finished on 60000/62483 queries. users per second: 98808.5

recommendations finished on 61000/62483 queries. users per second: 99081.3

recommendations finished on 62000/62483 queries. users per second: 99148

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |    226    |  0.7992424242424242 |  1   |
|     0      |    247    |  0.3333333333333335 |  2   |
|     0      |    230    | 0.32343749999999954 |  3   |
|     0      |    125    |  0.2666666666666663 |  4   |
|     0      |    294    |  0.2605042016806719 |  5   |
|     0      |    248    |         0.25        |  6   |
|     0      |    204    | 0.24249999999999988 |  7   |
|     0      |    155    |  0.2377358490566037 |  8   |
|     0      |    213    | 0.23548922056384758 |  9   |
|     0      |    276    | 0.23228346456692914 |  10  |
|     1      |    226    |  0.7992424242424242 |  1   |
|     1      |    247    |  0.3333333333333335 |  2   |
|     1      |    230    | 0.32343749999999954 |  3   |
|     1      |    125    |  0.2666666666666663 |  4   |
|     1      |    294    |  0.2605042016806719 |

In the result, although different models have different recommendation list, each user is recommended the same list of 10 items. This is because popularity is calculated by taking the most popular items across all users.

Collaborative Filtering Model

In [ ]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23257 users and 300 items.

Data prepared in: 0.135746s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 59000/62483 queries. users per second: 67474.4

recommendations finished on 60000/62483 queries. users per second: 67254.9

recommendations finished on 61000/62483 queries. users per second: 67294.2

recommendations finished on 62000/62483 queries. users per second: 67198.3

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |     2     | 0.15628153085708618 |  1   |
|     0      |     5     | 0.11865206406666683 |  2   |
|     0      |     21    | 0.10568000261600201 |  3   |
|     0      |     33    |  0.0915721975840055 |  4   |
|     0      |     15    | 0.07969538065103385 |  5   |
|     0      |     8     | 0.07838407846597525 |  6   |
|     0      |     11    | 0.07837480765122634 |  7   |
|     0      |     59    | 0.07703012686509353 |  8   |
|     0      |     0     | 0.07683671896274273 |  9   |
|     0      |     14    | 0.07090835846387424 |  10  |
|     1      |     2     | 0.10281421740849812 |  1   |
|     1      |     1     | 0.08884261382950677 |  2   |
|     1      |     21    | 0.07716303732660082 |  3   |
|     1      |     39    | 0.07691757877667744 |  4   |
|     1      |     5     | 0.07362190551227993 |

In [ ]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 100188 observations with 22999 users and 300 items.

Data prepared in: 0.146803s

...

recommendations finished on 59000/62483 queries. users per second: 66965.9

recommendations finished on 60000/62483 queries. users per second: 66917.5

recommendations finished on 61000/62483 queries. users per second: 66789.7

recommendations finished on 62000/62483 queries. users per second: 66599.8

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     0      |     2     | 0.06778916716575623  |  1   |
|     0      |     21    | 0.057318496704101565 |  2   |
|     0      |     14    | 0.05400232672691345  |  3   |
|     0      |     44    | 0.046806931495666504 |  4   |
|     0      |     4     | 0.04553281664848328  |  5   |
|     0      |     5     | 0.04395653605461121  |  6   |
|     0      |     16    | 0.043809282779693606 |  7   |
|     0      |     63    | 0.043410366773605345 |  8   |
|     0      |     24    |  0.0429409921169281  |  9   |
|     0      |     43    | 0.042276835441589354 |  10  |
|     1      |     11    | 0.08533799275755882  |  1   |
|     1      |     1     | 0.06907200440764427  |  2   |
|     1      |     14    | 0.051421333104372025 |  3   |
|     1      |     21    | 0.045379675924777985 |  4   |
|     1      |     5     | 0.04

In [ ]:
# Using normalized purchase count
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 100188 observations with 22877 users and 300 items.

Data prepared in: 0.134419s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 59000/62483 queries. users per second: 65181.1

recommendations finished on 60000/62483 queries. users per second: 65046.9

recommendations finished on 61000/62483 queries. users per second: 64956.4

recommendations finished on 62000/62483 queries. users per second: 64828.2

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|     0      |     2     |  0.005219560861587524 |  1   |
|     0      |     31    |  0.005169111490249634 |  2   |
|     0      |    248    |  0.005016016960144043 |  3   |
|     0      |     43    |  0.004192674160003662 |  4   |
|     0      |    139    |  0.003735482692718506 |  5   |
|     0      |     8     |  0.003126031160354614 |  6   |
|     0      |    137    | 0.0029234230518341065 |  7   |
|     0      |     30    | 0.0026917636394500734 |  8   |
|     0      |     73    | 0.0026587128639221193 |  9   |
|     0      |     13    |  0.002646172046661377 |  10  |
|     1      |     14    | 0.0059734702110290526 |  1   |
|     1      |    162    |  0.004814457893371582 |  2   |
|     1      |     8     |   0.0045684814453125  |  3   |
|     1      |     38    | 0.0035710811614990236 |  4   |
|     1      |

In [ ]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23257 users and 300 items.

Data prepared in: 0.13394s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 59000/62483 queries. users per second: 61617.3

recommendations finished on 60000/62483 queries. users per second: 61430.4

recommendations finished on 61000/62483 queries. users per second: 61095.8

recommendations finished on 62000/62483 queries. users per second: 60964.6

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |     34    | 3.065192854170405  |  1   |
|     0      |     37    | 3.0130212620966663 |  2   |
|     0      |    132    | 2.9815762309446736 |  3   |
|     0      |     0     |  2.9575444349735   |  4   |
|     0      |    248    | 2.9507693366530234 |  5   |
|     0      |     3     | 2.827047052444554  |  6   |
|     0      |    110    | 2.700192619287051  |  7   |
|     0      |     27    | 2.656566938631297  |  8   |
|     0      |     32    | 2.6142167654666277 |  9   |
|     0      |     10    | 2.606413994169095  |  10  |
|     1      |     34    | 3.066018771025149  |  1   |
|     1      |     37    | 3.0095574252800255 |  2   |
|     1      |    132    | 2.981114859663465  |  3   |
|     1      |     0     | 2.9568138195777336 |  4   |
|     1      |    248    | 2.954719705552589  |  5   |
|     1   

In [ ]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 100188 observations with 22999 users and 300 items.

Data prepared in: 0.130317s

...

recommendations finished on 59000/62483 queries. users per second: 82748.4

recommendations finished on 60000/62483 queries. users per second: 82670.9

recommendations finished on 61000/62483 queries. users per second: 82673.5

recommendations finished on 62000/62483 queries. users per second: 82491.3

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |    131    |  0.0  |  1   |
|     0      |    276    |  0.0  |  2   |
|     0      |    178    |  0.0  |  3   |
|     0      |    296    |  0.0  |  4   |
|     0      |    218    |  0.0  |  5   |
|     0      |    133    |  0.0  |  6   |
|     0      |    149    |  0.0  |  7   |
|     0      |    221    |  0.0  |  8   |
|     0      |     2     |  0.0  |  9   |
|     0      |     16    |  0.0  |  10  |
|     1      |     11    |  0.0  |  1   |
|     1      |    271    |  0.0  |  2   |
|     1      |    131    |  0.0  |  3   |
|     1      |    276    |  0.0  |  4   |
|     1      |    178    |  0.0  |  5   |
|     1      |    218    |  0.0  |  6   |
|     1      |    133    |  0.0  |  7   |
|     1      |    149    |  0.0  |  8   |
|     1      |    221    |  0.0  |  9   |
|     1      |     16    |  0.0  |  10  |
|     2      |    131    |  0.0  |

In [ ]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 100188 observations with 22877 users and 300 items.

Data prepared in: 0.134847s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 59000/62483 queries. users per second: 61321.6

recommendations finished on 60000/62483 queries. users per second: 61325.4

recommendations finished on 61000/62483 queries. users per second: 61178.8

recommendations finished on 62000/62483 queries. users per second: 60890.5

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |    226    |  0.7990779333042374 |  1   |
|     0      |    247    | 0.33331538637479147 |  2   |
|     0      |    230    |  0.3236202478408812 |  3   |
|     0      |    125    |  0.2666336278120676 |  4   |
|     0      |    294    | 0.26050272348548187 |  5   |
|     0      |    248    |  0.2534359395503998 |  6   |
|     0      |    204    |  0.2423662948608399 |  7   |
|     0      |    155    |  0.2377358490566038 |  8   |
|     0      |    213    |   0.23548006529041  |  9   |
|     0      |    276    |  0.2320187066953013 |  10  |
|     1      |    226    |  0.7993213169502487 |  1   |
|     1      |    247    | 0.33330311775207516 |  2   |
|     1      |    230    |  0.3233690857887267 |  3   |
|     1      |    125    | 0.26662703752517697 |  4   |
|     1      |    294    |  0.2603607929053427 |

In collaborative filtering above, we used two approaches: cosine and pearson distance. We also got to apply them to three training datasets with normal counts, dummy, or normalized counts of items purchase.
We can see that the recommendations are different for each user. This suggests that personalization does exist.
But how good is this model compared to the baseline..we will look upto evaluation for that

In [ ]:
#For evaluating recommendation engines
# RMSE (Root Mean Squared Errors)
# Measures the error of predicted values, Lesser the RMSE value, better the recommendations

# Recall
# What percentage of products that a user buys are actually recommended?
# If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6

# Precision
# Out of all the recommended items, how many the user actually liked?
# If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8

# Recall and precision both are important
# Consider a case where we recommend all products, so our customers will surely cover the items that they liked and bought. In this case, we have 100% recall!
# We have to consider precision. If we recommend 300 items but user likes and buys only 3 of them, then precision is 0.1%!
# This very low precision indicates that the model is not great, despite their excellent recall.
# So our aim has to be optimizing both recall and precision (to be close to 1 as possible).

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase CounTS']

Models on purchase counts

In [ ]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/14031 queries. users per second: 52485.2

recommendations finished on 2000/14031 queries. users per second: 52907.3

recommendations finished on 3000/14031 queries. users per second: 57111.3

recommendations finished on 4000/14031 queries. users per second: 59542.4

...

recommendations finished on 11000/14031 queries. users per second: 40404.6

recommendations finished on 12000/14031 queries. users per second: 40929

recommendations finished on 13000/14031 queries. users per second: 40062.5

recommendations finished on 14000/14031 queries. users per second: 39902.6


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |  0.00705580500320717  | 0.003883482531570038 |
|   2    |  0.005309671441807435 | 0.005764556392231114 |
|   3    |  0.004323759294894662 | 0.007011893637537142 |
|   4    |  0.007358705723041836 | 0.015175962642873046 |
|   5    |  0.006528401396906894 | 0.01730613811600334  |
|   6    |  0.006794478891977317 | 0.02164449094697971  |
|   7    |  0.006241282059114005 | 0.02307533625210339  |
|   8    | 0.0057818402109614435 | 0.024464423115318983 |
|   9    |  0.005606632931841459 | 0.027081435320123567 |
|   10   |  0.005430831729741287 |  0.0294810538518781  |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0634499113228957

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count 

Models on purchase dummy

In [ ]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/15565 queries. users per second: 65677.1

recommendations finished on 2000/15565 queries. users per second: 62642.9

recommendations finished on 3000/15565 queries. users per second: 62082.2

recommendations finished on 4000/15565 queries. users per second: 65817.6

...

recommendations finished on 12000/15565 queries. users per second: 54779.8

recommendations finished on 13000/15565 queries. users per second: 54936.5

recommendations finished on 14000/15565 queries. users per second: 55574.5

recommendations finished on 15000/15565 queries. users per second: 55876.1


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.007259877931256026 | 0.0031857160935220724 |
|   2    |  0.007356247992290406 |  0.005886886857654612 |
|   3    |  0.007152800085662257 |  0.008611558659422435 |
|   4    |  0.007131384516543545 |  0.01150553354986379  |
|   5    |  0.007105685833601082 |  0.014434360620997308 |
|   6    |  0.007281293500374828 |  0.018218211242303793 |
|   7    |  0.007287412234408674 |  0.021194926556077565 |
|   8    | 0.0070269836170896195 |  0.02349086909374835  |
|   9    |  0.006838705071920719 |  0.025848996207737934 |
|   10   |  0.006578862833279739 |  0.027552709027737276 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |

Models on normalized purchase frequency

In [ ]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)


PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/15590 queries. users per second: 62457.1

recommendations finished on 2000/15590 queries. users per second: 58227.6

recommendations finished on 3000/15590 queries. users per second: 59501.4

recommendations finished on 4000/15590 queries. users per second: 60391

...

recommendations finished on 12000/15590 queries. users per second: 39427.1

recommendations finished on 13000/15590 queries. users per second: 39593.9

recommendations finished on 14000/15590 queries. users per second: 39982.3

recommendations finished on 15000/15590 queries. users per second: 39814


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0030788967286722283 | 0.0014948023254630072 |
|   2    |  0.002918537524053881 |  0.002833928953236197 |
|   3    | 0.0028437032285653233 |  0.004529850568978215 |
|   4    | 0.0027581783194355358 |  0.005782671703133533 |
|   5    | 0.0027838357921744755 |  0.007548761076663582 |
|   6    |  0.002501603592046187 |  0.008108538054015922 |
|   7    | 0.0024282965270777963 |  0.008947674863325917 |
|   8    | 0.0023091725465041644 |  0.009676878843010968 |
|   9    | 0.0025158577435678182 |  0.01184322988203682  |
|   10   | 0.0025208466966004013 |  0.013043225809422089 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13426294993525362

Per User RMSE (best)
+------------+------+-------+
| customerId 

Model Selection

Since RMSE is higher using pearson distance than cosine, we would choose model the smaller mean squared errors, which in this case would be cosine. Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

Output : we will take output as an csv file, and will also build a function that will return recommendation list when given a customer ID.

In [ ]:
# We need to first rerun the model using the whole dataset,
# as we came to a final model using train data and evaluated with test set.
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy),
                                            user_id=user_id,
                                            item_id=item_id,
                                            target='purchase_dummy',
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.13949s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.092184s

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.10592188835144042  |  1   |
|    1553    |     1     | 0.10348175764083863  |  2   |
|    1553    |     35    |  0.0845762014389038  |  3   |
|    1553    |     33    |  0.0668614387512207  |  4   |
|    1553    |     5     | 0.06496070623397827  |  5   |
|    1553    |     61    | 0.060317397117614746 |  6   |
|    1553    |     15    | 0.05949603319168091  |  7   |
|    1553    |     17    |  0.0519999623298645  |  8   |
|    1553    |     11    | 0.050322222709655764 |  9   |
|    1553    |     19    | 0.049363481998443606 |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |    113    | 0.05312788486480713  |  3   |
|   20400    |     1     | 0.05210459232330322  |  4   |
|   20400    |     15    | 0.04

recommendations finished on 1000/1000 queries. users per second: 39710.9

In [ ]:
# creating an csv output file that will be saved in the drive
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('/content/drive/MyDrive/output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [ ]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 60897.6

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|247|230|125|294|248|204|155|213|276
11,226|247|230|125|294|248|204|155|276|74
12,226|247|230|125|294|248|204|155|213|276
16,226|247|230|125|294|248|204|155|213|276
21,226|247|230|125|294|248|204|155|213|276


In [ ]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [ ]:
customer_recomendation(21)

recommendedProducts    226|247|230|125|294|248|204|155|213|276
Name: 21, dtype: object

In [ ]:
customer_recomendation(2)

Customer not found.


2